# Library

In [1]:
import pandas as pd
import os
import numpy as np
from datetime import datetime
import pyexasol

In [2]:
pd.options.display.float_format = '{:.2f}'.format
# pd.reset_option('display.float_format')

In [3]:
# 경고문 없애는 코드
# UserWarning: Workbook contains no default style, apply openpyxl's default warn("Workbook contains no default style, apply openpyxl's default")
import warnings
warnings.simplefilter("ignore")

# Fold

In [4]:
pub_fold = 'D:/data/processing/big3/raw/public'
raw_fold = 'D:/data/processing/big3/raw'
raw2_fold = 'D:/data/processing/big3/raw/BD2'
raw4_fold = 'D:/data/processing/big3/raw/BD4'
raw5_fold = 'D:/data/processing/big3/raw/BD5'
an_fold = 'D:/data/processing/big3/analysis'
an2_fold = 'D:/data/processing/big3/analysis/BD2'
an4_fold = 'D:/data/processing/big3/analysis/BD4'
an5_fold = 'D:/data/processing/big3/analysis/BD5'
df_fold = 'D:/data/processing/big3/df'
df2_fold = 'D:/data/processing/big3/df/BD2'
df4_fold = 'D:/data/processing/big3/df/BD4'
df5_fold = 'D:/data/processing/big3/df/BD5'
folds = {
    'pub_fold':pub_fold, 
    'raw_fold':raw_fold, 
    'raw2_fold':raw2_fold, 
    'raw4_fold':raw4_fold, 
    'raw5_fold':raw5_fold, 
    'an_fold':an_fold, 
    'an2_fold:':an2_fold, 
    'an4_fold:':an4_fold, 
    'an5_fold:':an5_fold, 
    'df_fold':df_fold, 
    'df2_fold':df2_fold, 
    'df4_fold':df4_fold, 
    'df5_fold':df5_fold, 
    }
folds

{'pub_fold': 'D:/data/processing/big3/raw/public',
 'raw_fold': 'D:/data/processing/big3/raw',
 'raw2_fold': 'D:/data/processing/big3/raw/BD2',
 'raw4_fold': 'D:/data/processing/big3/raw/BD4',
 'raw5_fold': 'D:/data/processing/big3/raw/BD5',
 'an_fold': 'D:/data/processing/big3/analysis',
 'an2_fold:': 'D:/data/processing/big3/analysis/BD2',
 'an4_fold:': 'D:/data/processing/big3/analysis/BD4',
 'an5_fold:': 'D:/data/processing/big3/analysis/BD5',
 'df_fold': 'D:/data/processing/big3/df',
 'df2_fold': 'D:/data/processing/big3/df/BD2',
 'df4_fold': 'D:/data/processing/big3/df/BD4',
 'df5_fold': 'D:/data/processing/big3/df/BD5'}

# server

In [5]:
# exasol db
wd = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='VSYSD')
wz = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='VSYSZ')
we = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='VSYSE')

# 로드

## [로드]등록정보

In [57]:
# 5m 31.4s
car = wd.export_to_pandas("SELECT VIN, BSPL_STDG_CD, EXHST_GAS_GRD_CD, EXHST_GAS_CERT_NO, VHCL_ERSR_YN, MANP_MNG_NO, YRIDNW, VHCTY_CD, PURPS_CD2, FRST_REG_YMD, VHCL_FBCTN_YMD, VHCL_MNG_NO, VHRNO, EXTGAS_INSP_VLD_YMD, VHCL_OWNR_CL_CD FROM STD_CEG_CAR_MIG;")
car_ch_col = {
    'VIN':'차대번호', 
    'BSPL_STDG_CD':'법정동코드', 
    'EXHST_GAS_GRD_CD':'배출가스등급', 
    'EXHST_GAS_CERT_NO':'배출가스인증번호',
    'VHCL_ERSR_YN':'차량말소여부',
    'MANP_MNG_NO':'제원관리번호', 
    'YRIDNW':'차량연식', 
    'VHCTY_CD':'차종', 
    'PURPS_CD2':'용도', 
    'FRST_REG_YMD':'최초등록일자',
    'VHCL_FBCTN_YMD':'제작일자', 
    'VHCL_MNG_NO':'차량관리번호', 
    'VHRNO':'자동차등록번호',
    'EXTGAS_INSP_VLD_YMD':'검사유효일',
    'VHCL_OWNR_CL_CD':'소유자구분',  
}

print(car.shape)
print('data load : STD_CEG_CAR_MIG')

(31131025, 15)
data load : STD_CEG_CAR_MIG


In [58]:
carr = car.rename(columns=car_ch_col)
carr.columns

Index(['차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소여부', '제원관리번호', '차량연식', '차종',
       '용도', '최초등록일자', '제작일자', '차량관리번호', '자동차등록번호', '검사유효일', '소유자구분'],
      dtype='object')

In [59]:
print(carr['배출가스등급'].unique())
## 배출가스등급 코드 변환
grd_dict = {
    'A0501':'1', 
    'A0502':'2', 
    'A0503':'3', 
    'A0504':'4', 
    'A0505':'5', 
    'A05T2':'2',
    'A05T3':'3', 
    'A05T4':'4', 
    'A05T5':'5', 
    'A05X':'X', 
}
carr['배출가스등급'] = carr['배출가스등급'].replace(grd_dict)
carr['배출가스등급'] = carr['배출가스등급'].fillna('등급_결측값 or X')
print(carr['배출가스등급'].unique())

print(carr['차종'].unique())
# 4.0s
## 차종 코드 변환
cd_dict = {
    'A31M':'이륜', 
    'A31P':'승용', 
    'A31S':'특수', 
    'A31T':'화물', 
    'A31V':'승합'
}
carr['차종'] = carr['차종'].replace(cd_dict)
print(carr['차종'].unique())

print(carr['용도'].unique())
## 용도 코드 변환
purps_dict = {
    'A08P':'개인용', 
    'A08B':'영업용', 
    'A08O':'관용',
}
carr['용도'] = carr['용도'].replace(purps_dict)
print(carr['용도'].unique())

['A0503' 'A0502' 'A0505' 'A0501' 'A0504' nan]
['3' '2' '5' '1' '4' '등급_결측값 or X']
['A31P' 'A31T' 'A31V' 'A31S']
['승용' '화물' '승합' '특수']
['A08P' 'A08B' 'A08O']
['개인용' '영업용' '관용']


## [로드]제원정보

In [60]:
# 4.2s
src = wd.export_to_pandas("SELECT MANP_MNG_NO, FUEL_CD, VHCTY_TY_CD2, MNFCTR_NM, VHCNM, VHCL_FRM, EGIN_TY, VHCTY_CL_CD, TOTL_WGHT, CRYNG_WGHT, DSPLVL, EGIN_OTPT FROM STD_CEG_CAR_SRC_MIG;")
src_ch_col = {
    'MANP_MNG_NO':'제원관리번호', 
    'FUEL_CD':'연료',
    'VHCTY_TY_CD2':'차종유형', 
    'MNFCTR_NM':'제작사명', 
    'VHCNM':'차명', 
    'VHCL_FRM':'자동차형식', 
    'EGIN_TY':'엔진형식', 
    'VHCTY_CL_CD':'차종분류',
    'TOTL_WGHT':'총중량',
    'CRYNG_WGHT':'적재중량',
    'DSPLVL':'배기량', 
    'EGIN_OTPT':'엔진출력',
}
srcr = src.rename(columns=src_ch_col)

print(srcr.shape)
print('data load : STD_CEG_CAR_SRC_MIG')

(574467, 12)
data load : STD_CEG_CAR_SRC_MIG


In [66]:
print(srcr['연료'].unique())
# 7.9s
## 연료 코드 변환
fuel_dict = {
    'A90GS':'휘발유', 
    'A91DS':'경유',
    'A92LP':'LPG(액화석유가스)', 
    'A90GH':'휘발유 하이브리드', 
    'A93EV':'전기', 
    'A91DH':'경유 하이브리드', 
    'A92CN':'CNG(압축천연가스)', 
    'A93HD':'수소', 
    'A92LH':'LPG 하이브리드', 
    'A94OT':'기타연료', 
    'A92CH':'CNG 하이브리드',
    'A90AC':'알코올', 
    'A93SH':'태양열', 
    'A91KS':'등유', 
    'A92LN':'LNG(액화천연가스)', 
    'A90PH':'플러그인 하이브리드', 
}
srcr['연료'] = srcr['연료'].replace(fuel_dict)
srcr['연료'] = srcr['연료'].fillna('연료_결측값')
print(srcr['연료'].unique())

print(srcr['차종유형'].unique())
## 차종유형 코드 변환
ty_dict = {
    'A30C':'경형', 
    'A30L':'대형', 
    'A30M':'중형', 
    'A30S':'소형',
}
srcr['차종유형'] = srcr['차종유형'].replace(ty_dict)
srcr['차종유형'] = srcr['차종유형'].fillna('차종유형_결측값')
print(srcr['차종유형'].unique())

['휘발유' '기타연료' '등유' '전기' '경유' nan 'LPG(액화석유가스)' '휘발유 하이브리드' 'CNG(압축천연가스)'
 '알코올' '태양열' '경유 하이브리드' 'LPG 하이브리드' '수소' 'LNG(액화천연가스)' 'CNG 하이브리드']
['휘발유' '기타연료' '등유' '전기' '경유' '연료_결측값' 'LPG(액화석유가스)' '휘발유 하이브리드'
 'CNG(압축천연가스)' '알코올' '태양열' '경유 하이브리드' 'LPG 하이브리드' '수소' 'LNG(액화천연가스)'
 'CNG 하이브리드']
['대형' '중형' '소형' '경형' nan]
['대형' '중형' '소형' '경형' '차종유형_결측값']


## 법정동코드

In [62]:
# 1.3s
code = wd.export_to_pandas("SELECT STDG_CD, STDG_CTPV_NM, STDG_SGG_NM, STDG_CTPV_CD, STDG_SGG_CD FROM STD_BJCD_INFO;")
code_ch_col = {
    'STDG_CD':'법정동코드', 
    'STDG_CTPV_NM':'시도', 
    'STDG_SGG_NM':'시군구', 
    'STDG_CTPV_CD':'시도코드',
    'STDG_SGG_CD':'시군구코드', 
}
coder = code.rename(columns=code_ch_col)

print(coder.columns)
print(coder.shape)
print('data load : STD_BJCD_INFO')

Index(['법정동코드', '시도', '시군구', '시도코드', '시군구코드'], dtype='object')
(47916, 5)
data load : STD_BJCD_INFO


## [로드]LST

### from DB

In [63]:
# 7m 4.8s
# cs_raw = wz.export_to_pandas("SELECT * FROM VSYSZ.STD_CEG_GRD_LST WHERE CLS_YMD = (SELECT MAX(CLS_YMD) FROM VSYSZ.STD_CEG_GRD_LST);") # 최신 12월 31일자
cs_raw = wz.export_to_pandas("SELECT * FROM VSYSZ.STD_CEG_GRD_LST WHERE CLS_YMD = '20231130';") # 11월 30일자
cs_raw_dict = {
    'CLS_YMD':'원천데이터생성일자',

    'VIN':'차대번호', 
    'MANP_MNG_NO':'제원관리번호', 
    'VHCL_ERSR_YN':'차량말소여부',
    'FRST_REG_YMD':'최초등록일자',
    'YRIDNW':'연식', 
    'VHCTY_CD':'차종', 
    'PURPS_CD2':'용도', 
    'BSPL_STDG_CD':'법정동코드', 
    'EXHST_GAS_GRD_CD':'배출가스등급', 
    'FUEL_CD':'연료',
    'MNFCTR_NM':'제작사명', 
    'VHCTY_TY_CD2':'차종유형', 
    'VHCNM':'차명', 
    'VHCL_FRM':'자동차형식', 
    'EGIN_TY':'엔진형식', 
    'TOTL_WGHT':'총중량',
    'DSPLVL':'배기량', 
    'EGIN_OTPT':'엔진출력',

    'EXHST_GAS_CERT_NO':'배출가스인증번호',
    'VHCL_FBCTN_YMD':'제작일자', 
    'VHCL_MNG_NO':'차량관리번호', 
    'VHRNO':'자동차등록번호',
    'EXTGAS_INSP_VLD_YMD':'검사유효일',
    'VHCL_OWNR_CL_CD':'소유자구분',

    'VHCTY_CL_CD':'차종분류',
    'CRYNG_WGHT':'적재중량',

    'STDG_CTPV_NM':'시도명',
    'STDG_SGG_NM':'시군구명',
    'LEM_KND_CD':'저공해조치종류코드',
    'STRCT_CHG_YN':'구조변경여부',
    'FUEL_G_NM':'연료_G_명',
    'TKCAR_PSCP':'승차정원',
    'OVRLH':'전장',
    'OVRWD':'전폭',
    'OVRHG':'전고',
    'FBCTN_NTN_NM':'제작국명',
    'BOHUM_YN':'보훈여부',
}

cs = cs_raw.rename(columns=cs_raw_dict)

print(cs['원천데이터생성일자'].unique()[0])
print(cs.columns)
print(cs.shape)
print('data load : STD_CEG_GRD_LST')

20231130
Index(['원천데이터생성일자', '차량관리번호', '자동차등록번호', '법정동코드', '시도명', '시군구명', '소유자구분', '차종',
       '용도', '차대번호', '최초등록일자', '연식', '제작일자', '검사유효일', '배출가스인증번호', '배출가스등급',
       '저공해조치종류코드', '구조변경여부', '차량말소여부', '제원관리번호', '차명', '차종분류', '차종유형',
       '자동차형식', '제작사명', '연료', '연료_G_명', '엔진형식', '총중량', '적재중량', '엔진출력', '배기량',
       '승차정원', '전장', '전폭', '전고', '제작국명', '보훈여부'],
      dtype='object')
(31367557, 38)
data load : STD_CEG_GRD_LST


In [64]:
cs['배출가스등급'].unique()

array(['A0505', 'A0504', 'A0503', 'A0502', 'X', 'A0501'], dtype=object)

In [142]:
cs.drop_duplicates().shape

(31367557, 38)

#### [출력] STD_CEG_GRD_LST

In [71]:
today_date_dtl= datetime.today().strftime('%Y%m%d_%p_%I%M%S')
today_date_dtl

'20240105_PM_120814'

In [72]:
# # 3m 16.5s
# data_date = cs['원천데이터생성일자'].unique()[0]
# cs.to_csv(os.path.join(pub_fold, f'STD_CEG_GRD_LST_{data_date}_{today_date_dtl}.csv'), index=False, sep='|')

### from csv

In [73]:
# # 3m 15.1s
# cs_raw_dict = {
#     'CLS_YMD':'원천데이터생성일자',
    
#     'VIN':'차대번호', 
#     'MANP_MNG_NO':'제원관리번호', 
#     'VHCL_ERSR_YN':'차량말소여부',
#     'FRST_REG_YMD':'최초등록일자',
#     'YRIDNW':'연식', 
#     'VHCTY_CD':'차종', 
#     'PURPS_CD2':'용도', 
#     'BSPL_STDG_CD':'법정동코드', 
#     'EXHST_GAS_GRD_CD':'배출가스등급', 
#     'FUEL_CD':'연료',
#     'MNFCTR_NM':'제작사명', 
#     'VHCTY_TY_CD2':'차종유형', 
#     'VHCNM':'차명', 
#     'VHCL_FRM':'자동차형식', 
#     'EGIN_TY':'엔진형식', 
#     'TOTL_WGHT':'총중량',
#     'DSPLVL':'배기량', 
#     'EGIN_OTPT':'엔진출력',

#     'EXHST_GAS_CERT_NO':'배출가스인증번호',
#     'VHCL_FBCTN_YMD':'제작일자', 
#     'VHCL_MNG_NO':'차량관리번호', 
#     'VHRNO':'자동차등록번호',
#     'EXTGAS_INSP_VLD_YMD':'검사유효일',
#     'VHCL_OWNR_CL_CD':'소유자구분',

#     'VHCTY_CL_CD':'차종분류',
#     'CRYNG_WGHT':'적재중량',

#     'STDG_CTPV_NM':'시도명',
#     'STDG_SGG_NM':'시군구명',
#     'LEM_KND_CD':'저공해조치종류코드',
#     'STRCT_CHG_YN':'구조변경여부',
#     'FUEL_G_NM':'연료_G_명',
#     'TKCAR_PSCP':'승차정원',
#     'OVRLH':'전장',
#     'OVRWD':'전폭',
#     'OVRHG':'전고',
#     'FBCTN_NTN_NM':'제작국명',
#     'BOHUM_YN':'보훈여부',
# }

# name = 'STD_CEG_GRD_LST_20231231_20240105_AM_095035'
# file_name = f'{name}.csv'
# cs_raw = pd.read_csv(os.path.join(pub_fold, file_name), low_memory=False, usecols=cs_raw_dict.keys())
# cs = cs_raw.rename(columns=cs_raw_dict)
# print(cs.columns)
# print(cs.shape)
# print('data load : STD_CEG_GRD_LST')

In [67]:
# 31.8s
# 코드 변환
## 배출가스등급 코드 변환
print(cs['배출가스등급'].unique())
grd_dict = {
    'A0501':'1', 
    'A0502':'2', 
    'A0503':'3', 
    'A0504':'4', 
    'A0505':'5', 
    'A05T2':'2',
    'A05T3':'3', 
    'A05T4':'4', 
    'A05T5':'5', 
    'A05X':'X', 
    'X':'등급_결측값 or X'
}
cs['배출가스등급'] = cs['배출가스등급'].replace(grd_dict)
print(cs['배출가스등급'].unique())
print()

## 차종 코드 변환
print(cs['차종'].unique())
cd_dict = {
    'A31M':'이륜', 
    'A31P':'승용', 
    'A31S':'특수', 
    'A31T':'화물', 
    'A31V':'승합'
}
cs['차종'] = cs['차종'].replace(cd_dict)
print(cs['차종'].unique())
print()

## 용도 코드 변환
print(cs['용도'].unique())
purps_dict = {
    'A08P':'개인용', 
    'A08B':'영업용', 
    'A08O':'관용',
}
cs['용도'] = cs['용도'].replace(purps_dict)
cs['용도'].unique()
print()

## 소유자구분 코드 변환
print(cs['소유자구분'].unique())
ownr_dict = {
    # 'A27B':'사업자', 
    # 'A27F':'외국인', 
    # 'A27L':'법인', 
    # 'A27O':'기타', 
    # 'A27R':'주민', 

    'A08P':'개인용', 
    'A08B':'영업용', 
    'A08O':'관용', 
}
cs['소유자구분'] = cs['소유자구분'].replace(ownr_dict)
print(cs['소유자구분'].unique())
print()

## 연료 코드 변환
print(cs['연료'].unique())
fuel_dict = {
    'A90GS':'휘발유', 
    'A91DS':'경유',
    'A92LP':'LPG(액화석유가스)', 
    'A90GH':'휘발유 하이브리드', 
    'A93EV':'전기', 
    'A91DH':'경유 하이브리드', 
    'A92CN':'CNG(압축천연가스)', 
    'A93HD':'수소', 
    'A92LH':'LPG 하이브리드', 
    'A94OT':'기타연료', 
    'A92CH':'CNG 하이브리드',
    'A90AC':'알코올', 
    'A93SH':'태양열', 
    'A91KS':'등유', 
    'A92LN':'LNG(액화천연가스)', 
    'A90PH':'플러그인 하이브리드', 
}
cs['연료'] = cs['연료'].replace(fuel_dict)
cs['연료'] = cs['연료'].fillna('연료_결측값')
print(cs['연료'].unique())
print()

## 차종유형 코드 변환
print(cs['차종유형'].unique())
ty_dict = {
    'A30C':'경형', 
    'A30L':'대형', 
    'A30M':'중형', 
    'A30S':'소형',
}
cs['차종유형'] = cs['차종유형'].replace(ty_dict)
cs['차종유형'] = cs['차종유형'].fillna('차종유형_결측값')
print(cs['차종유형'].unique())

['5' '4' '3' '2' '등급_결측값 or X' '1']
['5' '4' '3' '2' '등급_결측값 or X' '1']

['화물' '승용' '승합' '특수']
['화물' '승용' '승합' '특수']

['개인용' '영업용' '관용']

['주민' '사업자' '법인' '외국인' '기타']
['주민' '사업자' '법인' '외국인' '기타']

['경유' '휘발유' 'LPG(액화석유가스)' nan '휘발유 하이브리드' '기타연료' 'CNG(압축천연가스)' '전기'
 'LPG 하이브리드' '경유 하이브리드' '알코올' '등유' '수소' '태양열' 'LNG(액화천연가스)' 'CNG 하이브리드']
['경유' '휘발유' 'LPG(액화석유가스)' '연료_결측값' '휘발유 하이브리드' '기타연료' 'CNG(압축천연가스)' '전기'
 'LPG 하이브리드' '경유 하이브리드' '알코올' '등유' '수소' '태양열' 'LNG(액화천연가스)' 'CNG 하이브리드']

['소형' '중형' '대형' '경형' nan]
['소형' '중형' '대형' '경형' '차종유형_결측값']


# 전처리

## 등록&제원

In [68]:
cs_before = carr.merge(srcr, on='제원관리번호', how='left')
cs_before.shape

(31131025, 26)

In [151]:
cs_before['연료'].unique()

array(['경유', '휘발유', 'LPG(액화석유가스)', '휘발유 하이브리드', '기타연료', '전기', '연료_결측값',
       'CNG(압축천연가스)', 'LPG 하이브리드', 'CNG 하이브리드', '수소', '경유 하이브리드', '등유',
       '태양열', 'LNG(액화천연가스)', nan, '알코올'], dtype=object)

In [152]:
cs_before['연료'] = cs_before['연료'].fillna('연료_결측값')

In [153]:
cs_before['연료'].unique()

array(['경유', '휘발유', 'LPG(액화석유가스)', '휘발유 하이브리드', '기타연료', '전기', '연료_결측값',
       'CNG(압축천연가스)', 'LPG 하이브리드', 'CNG 하이브리드', '수소', '경유 하이브리드', '등유',
       '태양열', 'LNG(액화천연가스)', '알코올'], dtype=object)

## 등록&제원&법정동

In [202]:
cs_before['법정동코드'].isnull().sum()

0

In [203]:
cs_before['법정동코드_수정'] = cs_before['법정동코드'].astype('str')
cs_before['법정동코드_수정'].head()

0    2714012300
1    2823710800
2    4111312700
3    3014011000
4    4713012500
Name: 법정동코드_수정, dtype: object

In [206]:
cs_before['법정동코드_수정'] = cs_before['법정동코드_수정'].str[:5] + '00000'
cs_before['법정동코드_수정'] = pd.to_numeric(cs_before['법정동코드_수정'], errors='coerce')
cs_before[['법정동코드', '법정동코드_수정']].head()

,법정동코드,법정동코드_수정
0,2714012300,2714000000
1,2823710800,2823700000
2,4111312700,4111300000
3,3014011000,3014000000
4,4713012500,4713000000


In [207]:
csc_before = cs_before.merge(coder, on='법정동코드', how='left')
csc_before.shape

(31131025, 31)

In [208]:
csc_before.columns

Index(['차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소여부', '제원관리번호', '차량연식', '차종',
       '용도', '최초등록일자', '제작일자', '차량관리번호', '자동차등록번호', '검사유효일', '소유자구분', '연료',
       '차종유형', '제작사명', '차명', '자동차형식', '엔진형식', '차종분류', '총중량', '적재중량', '배기량',
       '엔진출력', '법정동코드_수정', '시도', '시군구', '시도코드', '시군구코드'],
      dtype='object')

In [209]:
csc_before[['시도', '시군구']].isnull().sum()

시도     23
시군구    49
dtype: int64

In [210]:
csc_before.loc[csc_before['시도'].isnull(), '법정동코드']

255191      5180031025
6001670     5180031033
7154406     5180031025
7856130     4165052000
8692073     5180031028
12526574    5180031028
12635552    5172035023
13737539    5175037022
13840652    5172035030
14915888    5180031023
15418353    5180031027
16291706    5180031031
16758757    5180031031
18306064    5180031023
18315819    5180031023
18400019    5180031023
18680038    5180031031
23474359    5172035031
30252963    4163055000
30721007    5172035024
30936586    5172035024
31086014    5180031023
31125867    5180031031
Name: 법정동코드, dtype: int64

In [211]:
csc_before_codey = csc_before[csc_before['시도'].notnull()]
csc_before_coden = csc_before[csc_before['시도'].isnull()]
csc_before_codey.shape[0] + csc_before_coden.shape[0] == csc_before.shape[0]

True

In [212]:
csc_before_coden.shape

(23, 31)

In [213]:
csc_before_coden = csc_before_coden.drop(['법정동코드', '시도', '시군구', '시도코드', '시군구코드'], axis=1)
csc_before_coden.shape

(23, 26)

In [214]:
csc_before_codenm = csc_before_coden.merge(coder, left_on='법정동코드_수정', right_on='법정동코드', how='left')
csc_before_codenm.shape

(23, 31)

In [215]:
csc_before_codenm.columns

Index(['차대번호', '배출가스등급', '배출가스인증번호', '차량말소여부', '제원관리번호', '차량연식', '차종', '용도',
       '최초등록일자', '제작일자', '차량관리번호', '자동차등록번호', '검사유효일', '소유자구분', '연료', '차종유형',
       '제작사명', '차명', '자동차형식', '엔진형식', '차종분류', '총중량', '적재중량', '배기량', '엔진출력',
       '법정동코드_수정', '법정동코드', '시도', '시군구', '시도코드', '시군구코드'],
      dtype='object')

In [216]:
csc_before_codenm[['시도', '시군구']].isnull().sum()

시도     0
시군구    0
dtype: int64

In [217]:
csc_before = pd.concat([csc_before_codey, csc_before_codenm], ignore_index=True)
csc_before.shape # 31131025

(31131025, 31)

In [218]:
csc_before[['시도', '시군구']].isnull().sum()

시도      0
시군구    26
dtype: int64

In [219]:
csc_before.loc[csc_before['시군구'].isnull(), ['시도', '시군구']]

,시도,시군구
747535,세종특별자치시,NaN
770310,세종특별자치시,NaN
1641011,경기도,NaN
1800621,세종특별자치시,NaN
5992843,세종특별자치시,NaN
6854442,세종특별자치시,NaN
6886850,세종특별자치시,NaN
8007877,세종특별자치시,NaN
8385452,세종특별자치시,NaN
8961169,세종특별자치시,NaN


### [출력] 등록&제원

In [220]:
today_date_dtl = datetime.today().strftime('%Y%m%d_%p_%I%M%S')
today_date_dtl

'20240111_PM_014250'

In [221]:
# 2m 55.7s
csc_before.to_csv(os.path.join(pub_fold, f'등록_제원_법정동_전처리완료_{today_date_dtl}.csv'), index=False)

### [로드] 등록&제원 전처리 완료 

In [ ]:
# 2m 55.7s
csc_before = pd.read_csv(os.path.join(pub_fold, '등록_제원_법정동_전처리완료_20240105_PM_123234.csv'))
csc_before.shape

(31131025, 31)

In [87]:
csc_before['차량말소여부'].value_counts(dropna=False)

차량말소여부
N    25895547
Y     5235478
Name: count, dtype: int64

In [88]:
len(csc_before['시도'].unique())

17

## LST

### 시도명 수정
- 강원도 -> 강원특별자치도
- 제주도 -> 제주특별자치도

In [89]:
print(cs['시도명'].unique())

['전라남도' '대구광역시' '대전광역시' '강원도' '경상북도' '충청남도' '경상남도' '서울특별시' '부산광역시' '광주광역시'
 '경기도' '울산광역시' '제주특별자치도' '인천광역시' '충청북도' '전라북도' '세종특별자치시' '강원특별자치도']


In [90]:
cs.loc[cs['시도명'] == '강원도', '시도명'] = '강원특별자치도'
cs.loc[cs['시도명'] == '제주도', '시도명'] = '제주특별자치도'
print(cs['시도명'].unique())

['전라남도' '대구광역시' '대전광역시' '강원특별자치도' '경상북도' '충청남도' '경상남도' '서울특별시' '부산광역시'
 '광주광역시' '경기도' '울산광역시' '제주특별자치도' '인천광역시' '충청북도' '전라북도' '세종특별자치시']


In [91]:
len(cs['시도명'].unique())

17

### [출력] LST

In [92]:
today_date_dtl = datetime.today().strftime('%Y%m%d_%p_%I%M%S')
today_date_dtl

'20240110_PM_040426'

In [93]:
# 2m 55.3s
cs.to_csv(os.path.join(pub_fold, f'CEG_GRD_LST_전처리완료_{today_date_dtl}.csv'), index=False)

### [로드] LST 전처리 완료

In [9]:
# 2m 55.3s
cs = pd.read_csv(os.path.join(pub_fold, 'CEG_GRD_LST_전처리완료_20240105_PM_123234.csv'))
cs.shape

(31367557, 38)

In [94]:
cs['차량말소여부'].value_counts(dropna=False)

차량말소여부
N    25918267
Y     5449290
Name: count, dtype: int64

# 분석

In [222]:
csc_before['차량말소여부'].value_counts(dropna=False)

차량말소여부
N    25895547
Y     5235478
Name: count, dtype: int64

In [223]:
cs['차량말소여부'].value_counts(dropna=False)

차량말소여부
N    25918267
Y     5449290
Name: count, dtype: int64

In [224]:
csc_before.shape, cs.shape

((31131025, 31), (31367557, 38))

## 차량 현황
- 말소여부, 등급, 지역, 연료별 현황

In [225]:
csc_before['배출가스등급'].value_counts(dropna=False)

배출가스등급
2              12253589
3               9925044
1               3432331
5               2314350
4               1906910
등급_결측값 or X     1298801
Name: count, dtype: int64

### 등록&제원

In [226]:
# grp1_before = csc_before.groupby(['차량말소여부', '시도'], dropna=False)['차대번호'].count().reset_index()
# grp1_before.loc[grp1_before.shape[0]] = ['합계'] + grp1_before.iloc[:,1:].sum().to_list()
# grp1_before['합계'] = grp1_before.iloc[:, 1:].sum(axis=1)
# grp1_before # 31131025

In [227]:
grp1_before = csc_before.groupby(['차량말소여부', '시도'], dropna=False)['차대번호'].count().reset_index()
grp1_before.loc[grp1_before.shape[0], ['차량말소여부', '차대번호']] = ['합계', grp1_before['차대번호'].sum()]
grp1_before = grp1_before.rename(columns={'차대번호':'차량대수'})
grp1_before # 31131025

,차량말소여부,시도,차량대수
0,N,강원특별자치도,865371.00
1,N,경기도,6505925.00
2,N,경상남도,1938726.00
3,N,경상북도,1531987.00
4,N,광주광역시,721902.00
5,N,대구광역시,1256186.00
6,N,대전광역시,720579.00
7,N,부산광역시,1524100.00
8,N,서울특별시,3190988.00
9,N,세종특별자치시,197786.00


In [228]:
# grp2_before = csc_before.groupby(['차량말소여부', '배출가스등급'], dropna=False)['차대번호'].count().reset_index()
# grp2_before.columns = ['차량말소여부', '1', '2', '3', '4', '5', '등급_결측값']
# grp2_before.loc[grp2_before.shape[0]] = ['합계'] + grp2_before.iloc[:,1:].sum().to_list()
# grp2_before['합계'] = grp2_before.iloc[:, 1:].sum(axis=1)
# grp2_before # 31131025

In [229]:
grp2_before = csc_before.groupby(['차량말소여부', '배출가스등급'], dropna=False)['차대번호'].count().reset_index()
grp2_before.loc[grp2_before.shape[0], ['차량말소여부', '차대번호']] = ['합계', grp2_before['차대번호'].sum()]
grp2_before = grp2_before.rename(columns={'차대번호':'차량대수'})
grp2_before # 31131025

,차량말소여부,배출가스등급,차량대수
0,N,1,3260843.00
1,N,2,10929631.00
2,N,3,8185490.00
3,N,4,1335156.00
4,N,5,923641.00
5,N,등급_결측값 or X,1260786.00
6,Y,1,171488.00
7,Y,2,1323958.00
8,Y,3,1739554.00
9,Y,4,571754.00


In [230]:
# grp2_before.columns

In [231]:
# grp3_before = csc_before.groupby(['차량말소여부', '시도', '배출가스등급'], dropna=False)['차대번호'].count().unstack('배출가스등급').reset_index()
# grp3_before.columns = ['차량말소여부', '시도', '1', '2', '3', '4', '5', '등급_결측값']
# grp3_before.loc[grp3_before.shape[0], ['차량말소여부', '1', '2', '3', '4', '5', '등급_결측값']] = ['합계', grp3_before['1'].sum(), grp3_before['2'].sum(), grp3_before['3'].sum(), grp3_before['4'].sum(), grp3_before['5'].sum(), grp3_before['등급_결측값'].sum()]
# grp3_before['합계'] = grp3_before.iloc[:, 2:].sum(axis=1)
# grp3_before # 31131025

In [232]:
grp3_before = csc_before.groupby(['차량말소여부', '시도', '배출가스등급'], dropna=False)['차대번호'].count().reset_index()
grp3_before.loc[grp3_before.shape[0], ['차량말소여부', '차대번호']] = ['합계', grp3_before['차대번호'].sum()]
grp3_before = grp3_before.rename(columns={'차대번호':'차량대수'})
grp3_before # 31131025

,차량말소여부,시도,배출가스등급,차량대수
0,N,강원특별자치도,1,93671.00
1,N,강원특별자치도,2,353126.00
2,N,강원특별자치도,3,295897.00
3,N,강원특별자치도,4,48886.00
4,N,강원특별자치도,5,36684.00
...,...,...,...,...
200,Y,충청북도,3,60451.00
201,Y,충청북도,4,23292.00
202,Y,충청북도,5,72013.00
203,Y,충청북도,등급_결측값 or X,960.00


In [233]:
# grp4_before = csc_before.groupby(['차량말소여부', '연료', '배출가스등급'], dropna=False)['차대번호'].count().unstack('배출가스등급').reset_index()
# grp4_before.columns = ['차량말소여부', '연료', '1', '2', '3', '4', '5', '등급_결측값']
# grp4_before.loc[grp4_before.shape[0], ['차량말소여부', '1', '2', '3', '4', '5', '등급_결측값']] = ['합계', grp4_before['1'].sum(), grp4_before['2'].sum(), grp4_before['3'].sum(), grp4_before['4'].sum(), grp4_before['5'].sum(), grp4_before['등급_결측값'].sum()]
# grp4_before['합계'] = grp4_before.iloc[:, 2:].sum(axis=1)
# grp4_before # 31131025

In [263]:
csc_before['연료'].unique()

array(['경유', '휘발유', 'LPG(액화석유가스)', '휘발유 하이브리드', '기타연료', '전기', '연료_결측값',
       'CNG(압축천연가스)', 'LPG 하이브리드', 'CNG 하이브리드', '수소', '경유 하이브리드', '등유',
       '태양열', 'LNG(액화천연가스)', '알코올'], dtype=object)

In [234]:
grp4_before = csc_before.groupby(['차량말소여부', '연료', '배출가스등급'], dropna=False)['차대번호'].count().reset_index()
grp4_before.loc[grp4_before.shape[0], ['차량말소여부', '차대번호']] = ['합계', grp4_before['차대번호'].sum()]
grp4_before = grp4_before.rename(columns={'차대번호':'차량대수'})
grp4_before # 31131025

,차량말소여부,연료,배출가스등급,차량대수
0,N,CNG 하이브리드,1,269.00
1,N,CNG 하이브리드,2,17.00
2,N,CNG(압축천연가스),2,22139.00
3,N,CNG(압축천연가스),3,3905.00
4,N,CNG(압축천연가스),4,1.00
...,...,...,...,...
90,Y,휘발유,등급_결측값 or X,16962.00
91,Y,휘발유 하이브리드,1,33400.00
92,Y,휘발유 하이브리드,2,743.00
93,Y,휘발유 하이브리드,등급_결측값 or X,1954.00


In [235]:
grp4_before['차량말소여부'].unique(), grp4_before['연료'].unique(), grp4_before['배출가스등급'].unique()

(array(['N', 'Y', '합계'], dtype=object),
 array(['CNG 하이브리드', 'CNG(압축천연가스)', 'LNG(액화천연가스)', 'LPG 하이브리드',
        'LPG(액화석유가스)', '경유', '경유 하이브리드', '기타연료', '등유', '수소', '알코올',
        '연료_결측값', '전기', '태양열', '휘발유', '휘발유 하이브리드', nan], dtype=object),
 array(['1', '2', '3', '4', '5', '등급_결측값 or X', nan], dtype=object))

#### 출력-통계 등록&제원

In [182]:
today_date_dtl = datetime.today().strftime('%Y%m%d_%p_%I%M%S')
today_date_dtl

'20240110_PM_050427'

In [183]:
with pd.ExcelWriter(os.path.join(df_fold, f'기존등록제원_통계_{today_date_dtl}.xlsx')) as writer:
    grp1_before.to_excel(writer, sheet_name='말소_시도별', index=False)
    grp2_before.to_excel(writer, sheet_name='말소_등급별', index=False)
    grp3_before.to_excel(writer, sheet_name='말소_시도_등급별', index=False)
    grp4_before.to_excel(writer, sheet_name='말소_연료_등급별', index=False)

### LST

In [236]:
cs.shape

(31367557, 38)

In [237]:
# grp1 = cs.groupby(['차량말소여부', '시도명'], dropna=False)['차대번호'].count().unstack('시도명').reset_index()
# grp1.loc[grp1.shape[0]] = ['합계'] + grp1.iloc[:,1:].sum().to_list()
# grp1['합계'] = grp1.iloc[:, 1:].sum(axis=1)
# grp1 # 31367557

In [238]:
grp1 = cs.groupby(['차량말소여부', '시도명'], dropna=False)['차대번호'].count().reset_index()
grp1.loc[grp1.shape[0], ['차량말소여부', '차대번호']] = ['합계', grp1['차대번호'].sum()]
grp1 = grp1.rename(columns={'차대번호':'차량대수'})
grp1 # 31367557

,차량말소여부,시도명,차량대수
0,N,강원특별자치도,865992.00
1,N,경기도,6515691.00
2,N,경상남도,1941447.00
3,N,경상북도,1532871.00
4,N,광주광역시,722685.00
5,N,대구광역시,1256901.00
6,N,대전광역시,721575.00
7,N,부산광역시,1525701.00
8,N,서울특별시,3191576.00
9,N,세종특별자치시,197870.00


In [239]:
# grp2 = cs.groupby(['차량말소여부', '배출가스등급'], dropna=False)['차대번호'].count().unstack('배출가스등급').reset_index()
# grp2.columns = ['차량말소여부', '1', '2', '3', '4', '5', '등급_결측값']
# grp2.loc[grp2.shape[0]] = ['합계'] + grp2.iloc[:,1:].sum().to_list()
# grp2['합계'] = grp2.iloc[:, 1:].sum(axis=1)
# grp2 # 31367557

In [240]:
grp2 = cs.groupby(['차량말소여부', '배출가스등급'], dropna=False)['차대번호'].count().reset_index()
grp2.loc[grp2.shape[0], ['차량말소여부', '차대번호']] = ['합계', grp2['차대번호'].sum()]
grp2 = grp2.rename(columns={'차대번호':'차량대수'})
grp2 # 31367557

,차량말소여부,배출가스등급,차량대수
0,N,1,3277672.00
1,N,2,11661936.00
2,N,3,8550341.00
3,N,4,1328747.00
4,N,5,918555.00
5,N,등급_결측값 or X,181016.00
6,Y,1,176580.00
7,Y,2,1378219.00
8,Y,3,1802885.00
9,Y,4,595265.00


In [241]:
# grp3 = cs.groupby(['차량말소여부', '시도명', '배출가스등급'], dropna=False)['차대번호'].count().unstack('배출가스등급').reset_index()
# grp3.columns = ['차량말소여부', '시도명', '1', '2', '3', '4', '5', '등급_결측값']
# grp3.loc[grp3.shape[0], ['차량말소여부', '1', '2', '3', '4', '5', '등급_결측값']] = ['합계', grp3['1'].sum(), grp3['2'].sum(), grp3['3'].sum(), grp3['4'].sum(), grp3['5'].sum(), grp3['등급_결측값'].sum()]
# grp3['합계'] = grp3.iloc[:, 2:].sum(axis=1)
# grp3 # 31367557

In [242]:
grp3 = cs.groupby(['차량말소여부', '시도명', '배출가스등급'], dropna=False)['차대번호'].count().reset_index()
grp3.loc[grp3.shape[0], ['차량말소여부', '차대번호']] = ['합계', grp3['차대번호'].sum()]
grp3 = grp3.rename(columns={'차대번호':'차량대수'})
grp3 # 31367557

,차량말소여부,시도명,배출가스등급,차량대수
0,N,강원특별자치도,1,94265.00
1,N,강원특별자치도,2,371046.00
2,N,강원특별자치도,3,310482.00
3,N,강원특별자치도,4,48699.00
4,N,강원특별자치도,5,36427.00
...,...,...,...,...
200,Y,충청북도,3,60911.00
201,Y,충청북도,4,23525.00
202,Y,충청북도,5,72207.00
203,Y,충청북도,등급_결측값 or X,672.00


In [243]:
# grp4 = cs.groupby(['차량말소여부', '연료', '배출가스등급'], dropna=False)['차대번호'].count().unstack('배출가스등급').reset_index()
# grp4.columns = ['차량말소여부', '연료', '1', '2', '3', '4', '5', '등급_결측값']
# grp4.loc[grp4.shape[0], ['차량말소여부', '1', '2', '3', '4', '5', '등급_결측값']] = ['합계', grp4['1'].sum(), grp4['2'].sum(), grp4['3'].sum(), grp4['4'].sum(), grp4['5'].sum(), grp4['등급_결측값'].sum()]
# grp4['합계'] = grp4.iloc[:, 2:].sum(axis=1)
# grp4 # 31367557

In [244]:
grp4 = cs.groupby(['차량말소여부', '연료', '배출가스등급'], dropna=False)['차대번호'].count().reset_index()
grp4.loc[grp4.shape[0], ['차량말소여부', '차대번호']] = ['합계', grp4['차대번호'].sum()]
grp4 = grp4.rename(columns={'차대번호':'차량대수'})
grp4 # 31367557

,차량말소여부,연료,배출가스등급,차량대수
0,N,CNG 하이브리드,1,269.00
1,N,CNG 하이브리드,2,17.00
2,N,CNG(압축천연가스),2,24246.00
3,N,CNG(압축천연가스),3,3837.00
4,N,CNG(압축천연가스),4,1.00
...,...,...,...,...
79,Y,휘발유,등급_결측값 or X,19977.00
80,Y,휘발유 하이브리드,1,34173.00
81,Y,휘발유 하이브리드,2,2396.00
82,Y,휘발유 하이브리드,등급_결측값 or X,442.00


In [245]:
grp4['차량말소여부'].unique(), grp4['연료'].unique(), grp4['배출가스등급'].unique()

(array(['N', 'Y', '합계'], dtype=object),
 array(['CNG 하이브리드', 'CNG(압축천연가스)', 'LNG(액화천연가스)', 'LPG 하이브리드',
        'LPG(액화석유가스)', '경유', '경유 하이브리드', '기타연료', '등유', '수소', '알코올',
        '연료_결측값', '전기', '태양열', '휘발유', '휘발유 하이브리드', nan], dtype=object),
 array(['1', '2', '3', '4', '5', '등급_결측값 or X', nan], dtype=object))

In [246]:
grp4_before['차량말소여부'].unique(), grp4_before['연료'].unique(), grp4_before['배출가스등급'].unique()

(array(['N', 'Y', '합계'], dtype=object),
 array(['CNG 하이브리드', 'CNG(압축천연가스)', 'LNG(액화천연가스)', 'LPG 하이브리드',
        'LPG(액화석유가스)', '경유', '경유 하이브리드', '기타연료', '등유', '수소', '알코올',
        '연료_결측값', '전기', '태양열', '휘발유', '휘발유 하이브리드', nan], dtype=object),
 array(['1', '2', '3', '4', '5', '등급_결측값 or X', nan], dtype=object))

#### 출력-통계 LST

In [119]:
today_date_dtl = datetime.today().strftime('%Y%m%d_%p_%I%M%S')
today_date_dtl

'20240110_PM_041029'

In [120]:
with pd.ExcelWriter(os.path.join(df_fold, f'RH제공파일_통계_{today_date_dtl}.xlsx')) as writer:
    grp1.to_excel(writer, sheet_name='말소_시도별', index=False)
    grp2.to_excel(writer, sheet_name='말소_등급별', index=False)
    grp3.to_excel(writer, sheet_name='말소_시도_등급별', index=False)
    grp4.to_excel(writer, sheet_name='말소_연료_등급별', index=False)

### 등록&제원과 LST 통계 비교

#### 1\. 말소, 시도별 차량대수

In [247]:
grp1 = grp1.rename(columns={'시도명':'시도'})

In [248]:
grp1_before['시도'].unique(), grp1['시도'].unique()

(array(['강원특별자치도', '경기도', '경상남도', '경상북도', '광주광역시', '대구광역시', '대전광역시',
        '부산광역시', '서울특별시', '세종특별자치시', '울산광역시', '인천광역시', '전라남도', '전라북도',
        '제주특별자치도', '충청남도', '충청북도', nan], dtype=object),
 array(['강원특별자치도', '경기도', '경상남도', '경상북도', '광주광역시', '대구광역시', '대전광역시',
        '부산광역시', '서울특별시', '세종특별자치시', '울산광역시', '인천광역시', '전라남도', '전라북도',
        '제주특별자치도', '충청남도', '충청북도', nan], dtype=object))

In [249]:
len(grp1_before['시도'].unique()), len(grp1['시도'].unique())

(18, 18)

In [250]:
len(set(grp1_before['시도'].unique()) & set(grp1['시도'].unique()))

18

In [251]:
grp1_before.columns, grp1.columns

(Index(['차량말소여부', '시도', '차량대수'], dtype='object'),
 Index(['차량말소여부', '시도', '차량대수'], dtype='object'))

In [252]:
tgrp1 = grp1_before.merge(grp1, on=['차량말소여부', '시도'], how='outer')
tgrp1['차이(A-B)'] = tgrp1['차량대수_x'] - tgrp1['차량대수_y']
tgrp1 = tgrp1.rename(columns={'차량대수_x':'기존(A)', '차량대수_y':'RH(B)'})
tgrp1

,차량말소여부,시도,기존(A),RH(B),차이(A-B)
0,N,강원특별자치도,865371.00,865992.00,-621.00
1,N,경기도,6505925.00,6515691.00,-9766.00
2,N,경상남도,1938726.00,1941447.00,-2721.00
3,N,경상북도,1531987.00,1532871.00,-884.00
4,N,광주광역시,721902.00,722685.00,-783.00
5,N,대구광역시,1256186.00,1256901.00,-715.00
6,N,대전광역시,720579.00,721575.00,-996.00
7,N,부산광역시,1524100.00,1525701.00,-1601.00
8,N,서울특별시,3190988.00,3191576.00,-588.00
9,N,세종특별자치시,197786.00,197870.00,-84.00


#### 2\. 말소여부, 등급별 차량대수

In [253]:
grp2_before.columns, grp2.columns

(Index(['차량말소여부', '배출가스등급', '차량대수'], dtype='object'),
 Index(['차량말소여부', '배출가스등급', '차량대수'], dtype='object'))

In [254]:
grp2_before['배출가스등급'].unique(), grp2['배출가스등급'].unique()

(array(['1', '2', '3', '4', '5', '등급_결측값 or X', nan], dtype=object),
 array(['1', '2', '3', '4', '5', '등급_결측값 or X', nan], dtype=object))

In [255]:
tgrp2 = grp2_before.merge(grp2, on=['차량말소여부', '배출가스등급'], how='outer')
tgrp2['차이(A-B)'] = tgrp2['차량대수_x'] - tgrp2['차량대수_y']
tgrp2 = tgrp2.rename(columns={'차량대수_x':'기존(A)', '차량대수_y':'RH(B)'})
tgrp2

,차량말소여부,배출가스등급,기존(A),RH(B),차이(A-B)
0,N,1,3260843.00,3277672.00,-16829.00
1,N,2,10929631.00,11661936.00,-732305.00
2,N,3,8185490.00,8550341.00,-364851.00
3,N,4,1335156.00,1328747.00,6409.00
4,N,5,923641.00,918555.00,5086.00
5,N,등급_결측값 or X,1260786.00,181016.00,1079770.00
6,Y,1,171488.00,176580.00,-5092.00
7,Y,2,1323958.00,1378219.00,-54261.00
8,Y,3,1739554.00,1802885.00,-63331.00
9,Y,4,571754.00,595265.00,-23511.00


#### 3\. 말소여부, 시도, 등급별 차량대수

In [256]:
grp3_before.columns, grp3.columns

(Index(['차량말소여부', '시도', '배출가스등급', '차량대수'], dtype='object'),
 Index(['차량말소여부', '시도명', '배출가스등급', '차량대수'], dtype='object'))

In [257]:
grp3 = grp3.rename(columns={'시도명':'시도'})

In [258]:
tgrp3 = grp3_before.merge(grp3, on=['차량말소여부', '시도', '배출가스등급'], how='outer')
tgrp3['차이(A-B)'] = tgrp3['차량대수_x'] - tgrp3['차량대수_y']
tgrp3 = tgrp3.rename(columns={'차량대수_x':'기존(A)', '차량대수_y':'RH(B)'})
tgrp3

,차량말소여부,시도,배출가스등급,기존(A),RH(B),차이(A-B)
0,N,강원특별자치도,1,93671.00,94265.00,-594.00
1,N,강원특별자치도,2,353126.00,371046.00,-17920.00
2,N,강원특별자치도,3,295897.00,310482.00,-14585.00
3,N,강원특별자치도,4,48886.00,48699.00,187.00
4,N,강원특별자치도,5,36684.00,36427.00,257.00
...,...,...,...,...,...,...
200,Y,충청북도,3,60451.00,60911.00,-460.00
201,Y,충청북도,4,23292.00,23525.00,-233.00
202,Y,충청북도,5,72013.00,72207.00,-194.00
203,Y,충청북도,등급_결측값 or X,960.00,672.00,288.00


#### 4\. 말소여부, 연료, 등급별 차량대수

In [259]:
grp4_before.columns, grp4.columns

(Index(['차량말소여부', '연료', '배출가스등급', '차량대수'], dtype='object'),
 Index(['차량말소여부', '연료', '배출가스등급', '차량대수'], dtype='object'))

In [260]:
grp4_before.shape, grp4.shape

((95, 4), (84, 4))

In [261]:
grp4_before['연료'].unique(), grp4['연료'].unique()

(array(['CNG 하이브리드', 'CNG(압축천연가스)', 'LNG(액화천연가스)', 'LPG 하이브리드',
        'LPG(액화석유가스)', '경유', '경유 하이브리드', '기타연료', '등유', '수소', '알코올',
        '연료_결측값', '전기', '태양열', '휘발유', '휘발유 하이브리드', nan], dtype=object),
 array(['CNG 하이브리드', 'CNG(압축천연가스)', 'LNG(액화천연가스)', 'LPG 하이브리드',
        'LPG(액화석유가스)', '경유', '경유 하이브리드', '기타연료', '등유', '수소', '알코올',
        '연료_결측값', '전기', '태양열', '휘발유', '휘발유 하이브리드', nan], dtype=object))

In [262]:
tgrp4 = grp4_before.merge(grp4, on=['차량말소여부', '연료', '배출가스등급'], how='outer')
tgrp4['차이(A-B)'] = tgrp4['차량대수_x'] - tgrp4['차량대수_y']
tgrp4 = tgrp4.rename(columns={'차량대수_x':'기존(A)', '차량대수_y':'RH(B)'})
tgrp4

,차량말소여부,연료,배출가스등급,기존(A),RH(B),차이(A-B)
0,N,CNG 하이브리드,1,269.00,269.00,0.00
1,N,CNG 하이브리드,2,17.00,17.00,0.00
2,N,CNG(압축천연가스),2,22139.00,24246.00,-2107.00
3,N,CNG(압축천연가스),3,3905.00,3837.00,68.00
4,N,CNG(압축천연가스),4,1.00,1.00,0.00
...,...,...,...,...,...,...
92,Y,휘발유 하이브리드,2,743.00,2396.00,-1653.00
93,Y,휘발유 하이브리드,등급_결측값 or X,1954.00,442.00,1512.00
94,합계,NaN,NaN,31131025.00,31367557.00,-236532.00
95,N,LPG 하이브리드,2,NaN,1.00,NaN


In [264]:
tgrp4[tgrp4['연료'] == '연료_결측값']

,차량말소여부,연료,배출가스등급,기존(A),RH(B),차이(A-B)
32,N,연료_결측값,2,21.00,NaN,NaN
33,N,연료_결측값,3,96.00,NaN,NaN
34,N,연료_결측값,4,48.00,NaN,NaN
35,N,연료_결측값,5,21.00,NaN,NaN
36,N,연료_결측값,등급_결측값 or X,76627.00,150423.00,-73796.00
74,Y,연료_결측값,1,1.00,NaN,NaN
75,Y,연료_결측값,2,21.00,NaN,NaN
76,Y,연료_결측값,3,138.00,NaN,NaN
77,Y,연료_결측값,4,12.00,NaN,NaN
78,Y,연료_결측값,5,2.00,NaN,NaN


#### 출력-통계 기존_LST

In [265]:
tgrp4[tgrp4['연료'].isnull()]

,차량말소여부,연료,배출가스등급,기존(A),RH(B),차이(A-B)
94,합계,NaN,NaN,31131025.00,31367557.00,-236532.00


In [266]:
today_date_dtl = datetime.today().strftime('%Y%m%d_%p_%I%M%S')
today_date_dtl

'20240115_PM_124654'

In [267]:
with pd.ExcelWriter(os.path.join(df_fold, f'기존_RH_비교통계_{today_date_dtl}.xlsx')) as writer:
    tgrp1.to_excel(writer, sheet_name='말소_시도별', index=False)
    tgrp2.to_excel(writer, sheet_name='말소_등급별', index=False)
    tgrp3.to_excel(writer, sheet_name='말소_시도_등급별', index=False)
    tgrp4.to_excel(writer, sheet_name='말소_연료_등급별', index=False)

# code end